In [173]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from pandas.api.types import is_numeric_dtype
import matplotlib.pyplot as plt

np.set_printoptions(precision=4)
orig = pd.read_csv('weatherAUS.csv')

In [174]:
orig.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [175]:
print(orig.shape)

(145460, 23)


In [176]:
print(orig.isna().sum())

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64


In [177]:
df = orig.copy()

# YES/NO => 1/0
df.RainTomorrow = df.RainTomorrow.map(lambda e: e == 'Yes').astype(int)
df.RainToday = df.RainToday.map(lambda e: e == 'Yes').astype(int)

# "2020-03-04" => [3], [4]
month = df.Date.map(lambda e : e[5:7]).astype(int)
date = df.Date.map(lambda e : e[8:]).astype(int)
df.insert(value=month, column='month', loc=0)
df.Date = date
df = df.drop(columns=['Date', 'Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm'])

df.head()

,month,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,12,Albury,13.4,22.9,0.6,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,16.9,21.8,0,0
1,12,Albury,7.4,25.1,0.0,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,17.2,24.3,0,0
2,12,Albury,12.9,25.7,0.0,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,21.0,23.2,0,0
3,12,Albury,9.2,28.0,0.0,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,18.1,26.5,0,0
4,12,Albury,17.5,32.3,1.0,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,17.8,29.7,0,0


In [178]:
df = df.dropna()
df.shape

(113679, 19)

In [179]:
# map string values to int enums
labelEncoders = {}

# df['thal'] = pd.Categorical(df['thal'])
# df['thal'] = df.thal.cat.codes

for col in df.columns:
    if not is_numeric_dtype(df[col].dtype):
        le = LabelEncoder()
        le.fit(df[col])
        df[col] = le.transform(df[col])

        # print(col, le.classes_)
        labelEncoders[col] = le

    df[col] = df[col].fillna(0)

df.head()

,month,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,12,1,13.4,22.9,0.6,13,44.0,13,14,20.0,24.0,71.0,22.0,1007.7,1007.1,16.9,21.8,0,0
1,12,1,7.4,25.1,0.0,14,44.0,6,15,4.0,22.0,44.0,25.0,1010.6,1007.8,17.2,24.3,0,0
2,12,1,12.9,25.7,0.0,15,46.0,13,15,19.0,26.0,38.0,30.0,1007.6,1008.7,21.0,23.2,0,0
3,12,1,9.2,28.0,0.0,4,24.0,9,0,11.0,9.0,45.0,16.0,1017.6,1012.8,18.1,26.5,0,0
4,12,1,17.5,32.3,1.0,13,41.0,1,7,7.0,20.0,82.0,33.0,1010.8,1006.0,17.8,29.7,0,0


# Feature Corelation

`sns.pairplot(df[['Rainfall', 'Humidity9am', 'Humidity3pm']], diag_kind='kde')`

## Humidity

![rainfall-humidity](rainfall-humidity.png)

## Pressure

![rainfall-presure](rainfall-presure.png)

## Temprature

![rainfall-temp](rainfall-temp.png)

## Wind Direction

![rainfall-winddir](rainfall-winddir.png)

## Wind Speed

![rainfall-windspeed](rainfall-windspeed.png)


In [180]:
df = df.drop(columns=['WindGustDir', 'WindDir9am', 'WindDir3pm'])

In [181]:
# Train a decision tree to check feature importance
forest = ExtraTreesClassifier(n_estimators=50,
                              random_state=0)

X = df.drop(columns=['RainTomorrow'])
y = df.RainTomorrow

forest.fit(X, y)


ExtraTreesClassifier(n_estimators=50, random_state=0)

In [182]:
importances = forest.feature_importances_

std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)

indices = np.argsort(importances)[::-1] # desc

featureCnt = X.shape[1]

impDetails = []

for i in range(featureCnt):
    featureName = df.columns[i]
    impDetails.append([i + 1, featureName, importances[indices[i]]])
    
ftImp = pd.DataFrame(data=impDetails, columns=['Ranking', 'Feature', 'Importance'])

ftImp

,Ranking,Feature,Importance
0,1,month,0.188893
1,2,Location,0.075742
2,3,MinTemp,0.073664
3,4,MaxTemp,0.072594
4,5,Rainfall,0.068948
5,6,WindGustSpeed,0.064376
6,7,WindSpeed9am,0.057090
7,8,WindSpeed3pm,0.056547
8,9,Humidity9am,0.054527
9,10,Humidity3pm,0.052349


In [183]:
from sklearn.neural_network import MLPClassifier

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size=0.1, random_state=1)

print('train: ', X_train.shape)
print('validation: ', X_val.shape)
print('test: ', X_test.shape)

clf = MLPClassifier(alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train, Y_train)

train:  (90943, 15)
validation:  (2274, 15)
test:  (20462, 15)


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1)

In [184]:
y_predict = clf.predict(X_test)
accuracy_score(Y_test, Y_predict)

0.8451764245919265

In [192]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.layers.experimental import preprocessing


In [186]:
X_train.describe().transpose()[['mean', 'std']]

,mean,std
month,6.436119,3.475043
Location,21.313966,12.800596
MinTemp,12.668686,6.248845
MaxTemp,23.661835,6.985262
Rainfall,2.360443,8.471161
WindGustSpeed,40.797741,13.314615
WindSpeed9am,15.186369,8.353468
WindSpeed3pm,19.517412,8.576278
Humidity9am,67.372365,18.914013
Humidity3pm,50.635387,20.745952


In [223]:
normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(X_train))

# print(normalizer.mean.numpy())

first = np.array(train_features[:1])

with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())

First example: [[   5.     6.    13.2   23.     0.    24.     7.     6.    77.    58.
  1011.1 1008.8   17.3   21.7    0. ]]

Normalized: [[-0.41 -1.2   0.09 -0.09 -0.28 -1.26 -0.98 -1.58  0.51  0.35 -0.9  -0.9
  -0.03 -0.06 -0.54]]


In [230]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(
      optimizer=tfa.optimizers.RectifiedAdam(0.00001),
      loss=tf.keras.losses.BinaryCrossentropy(from_logits=False, label_smoothing=0, name='binary_crossentropy'),
      metrics=['accuracy'])
  return model

In [ ]:
model = get_compiled_model()
model.fit(X_train, Y_train, epochs=300, batch_size=2048, validation_data=(X_val, Y_val))

model.summary()

Epoch 1/300
45/45 [==============================] - 2s 6ms/step - loss: 1.5848 - accuracy: 0.2194 - val_loss: 1.5662 - val_accuracy: 0.2234
Epoch 2/300
45/45 [==============================] - 0s 3ms/step - loss: 1.5685 - accuracy: 0.2193 - val_loss: 1.5530 - val_accuracy: 0.2234
Epoch 3/300
45/45 [==============================] - 0s 2ms/step - loss: 1.5530 - accuracy: 0.2196 - val_loss: 1.5358 - val_accuracy: 0.2234
Epoch 4/300
45/45 [==============================] - 0s 3ms/step - loss: 1.5327 - accuracy: 0.2209 - val_loss: 1.5154 - val_accuracy: 0.2234
Epoch 5/300
45/45 [==============================] - 0s 3ms/step - loss: 1.5141 - accuracy: 0.2200 - val_loss: 1.4924 - val_accuracy: 0.2234
Epoch 6/300
45/45 [==============================] - 0s 3ms/step - loss: 1.4912 - accuracy: 0.2196 - val_loss: 1.4669 - val_accuracy: 0.2234
Epoch 7/300
45/45 [==============================] - 0s 2ms/step - loss: 1.4618 - accuracy: 0.2204 - val_loss: 1.4387 - val_accuracy: 0.2234
Epoch 8/300
4

45/45 [==============================] - 0s 2ms/step - loss: 0.5090 - accuracy: 0.7838 - val_loss: 0.5183 - val_accuracy: 0.7801
Epoch 59/300
45/45 [==============================] - 0s 2ms/step - loss: 0.5091 - accuracy: 0.7839 - val_loss: 0.5180 - val_accuracy: 0.7801
Epoch 60/300
45/45 [==============================] - 0s 2ms/step - loss: 0.5094 - accuracy: 0.7830 - val_loss: 0.5178 - val_accuracy: 0.7801
Epoch 61/300
45/45 [==============================] - 0s 2ms/step - loss: 0.5094 - accuracy: 0.7832 - val_loss: 0.5176 - val_accuracy: 0.7801
Epoch 62/300
45/45 [==============================] - 0s 2ms/step - loss: 0.5110 - accuracy: 0.7812 - val_loss: 0.5173 - val_accuracy: 0.7801
Epoch 63/300
45/45 [==============================] - 0s 2ms/step - loss: 0.5100 - accuracy: 0.7830 - val_loss: 0.5171 - val_accuracy: 0.7801
Epoch 64/300
45/45 [==============================] - 0s 2ms/step - loss: 0.5096 - accuracy: 0.7829 - val_loss: 0.5168 - val_accuracy: 0.7801
Epoch 65/300
45/45 

Epoch 116/300
45/45 [==============================] - 0s 2ms/step - loss: 0.4893 - accuracy: 0.7834 - val_loss: 0.4955 - val_accuracy: 0.7810
Epoch 117/300
45/45 [==============================] - 0s 2ms/step - loss: 0.4868 - accuracy: 0.7842 - val_loss: 0.4949 - val_accuracy: 0.7810
Epoch 118/300
45/45 [==============================] - 0s 2ms/step - loss: 0.4881 - accuracy: 0.7824 - val_loss: 0.4943 - val_accuracy: 0.7810
Epoch 119/300
45/45 [==============================] - 0s 2ms/step - loss: 0.4855 - accuracy: 0.7842 - val_loss: 0.4936 - val_accuracy: 0.7810
Epoch 120/300
45/45 [==============================] - 0s 2ms/step - loss: 0.4849 - accuracy: 0.7851 - val_loss: 0.4929 - val_accuracy: 0.7810
Epoch 121/300
45/45 [==============================] - 0s 2ms/step - loss: 0.4828 - accuracy: 0.7858 - val_loss: 0.4922 - val_accuracy: 0.7810
Epoch 122/300
45/45 [==============================] - 0s 2ms/step - loss: 0.4855 - accuracy: 0.7836 - val_loss: 0.4915 - val_accuracy: 0.7810

45/45 [==============================] - 0s 2ms/step - loss: 0.4456 - accuracy: 0.7914 - val_loss: 0.4521 - val_accuracy: 0.7889
Epoch 174/300
45/45 [==============================] - 0s 2ms/step - loss: 0.4443 - accuracy: 0.7911 - val_loss: 0.4512 - val_accuracy: 0.7894
Epoch 175/300
45/45 [==============================] - 0s 2ms/step - loss: 0.4453 - accuracy: 0.7908 - val_loss: 0.4504 - val_accuracy: 0.7894
Epoch 176/300
45/45 [==============================] - 0s 2ms/step - loss: 0.4421 - accuracy: 0.7930 - val_loss: 0.4496 - val_accuracy: 0.7902
Epoch 177/300
45/45 [==============================] - 0s 2ms/step - loss: 0.4428 - accuracy: 0.7945 - val_loss: 0.4487 - val_accuracy: 0.7907
Epoch 178/300
45/45 [==============================] - 0s 2ms/step - loss: 0.4416 - accuracy: 0.7930 - val_loss: 0.4480 - val_accuracy: 0.7902
Epoch 179/300
45/45 [==============================] - 0s 2ms/step - loss: 0.4425 - accuracy: 0.7919 - val_loss: 0.4471 - val_accuracy: 0.7907
Epoch 180/300

In [232]:
result = model.evaluate(X_test, Y_test)
dict(zip(model.metrics_names, result))

640/640 [==============================] - 0s 580us/step - loss: 0.4461 - accuracy: 0.8058


{'loss': 0.4460500478744507, 'accuracy': 0.8058351874351501}